In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('./my_data/my_data/german_credit_data_dataset.csv')
data.head(10)

,checking_account_status,duration,credit_history,purpose,credit_amount,savings,present_employment,installment_rate,personal,other_debtors,...,property,age,other_installment_plans,housing,existing_credits,job,dependents,telephone,foreign_worker,customer_type
0,A11,6,A34,A43,1169.0,A65,A75,4.0,A93,A101,...,A121,67.0,A143,A152,2.0,A173,1,A192,A201,1
1,A12,48,A32,A43,5951.0,A61,A73,2.0,A92,A101,...,A121,22.0,A143,A152,1.0,A173,1,A191,A201,2
2,A14,12,A34,A46,2096.0,A61,A74,2.0,A93,A101,...,A121,49.0,A143,A152,1.0,A172,2,A191,A201,1
3,A11,42,A32,A42,7882.0,A61,A74,2.0,A93,A103,...,A122,45.0,A143,A153,1.0,A173,2,A191,A201,1
4,A11,24,A33,A40,4870.0,A61,A73,3.0,A93,A101,...,A124,53.0,A143,A153,2.0,A173,2,A191,A201,2
5,A14,36,A32,A46,9055.0,A65,A73,2.0,A93,A101,...,A124,35.0,A143,A153,1.0,A172,2,A192,A201,1
6,A14,24,A32,A42,2835.0,A63,A75,3.0,A93,A101,...,A122,53.0,A143,A152,1.0,A173,1,A191,A201,1
7,A12,36,A32,A41,6948.0,A61,A73,2.0,A93,A101,...,A123,35.0,A143,A151,1.0,A174,1,A192,A201,1
8,A14,12,A32,A43,3059.0,A64,A74,2.0,A91,A101,...,A121,61.0,A143,A152,1.0,A172,1,A191,A201,1
9,A12,30,A34,A40,5234.0,A61,A71,4.0,A94,A101,...,A123,28.0,A143,A152,2.0,A174,1,A191,A201,2


## 查看缺失值

In [3]:
data.isnull().sum().sort_values(ascending=False)
#check false value

checking_account_status    0
property                   0
foreign_worker             0
telephone                  0
dependents                 0
job                        0
existing_credits           0
housing                    0
other_installment_plans    0
age                        0
present_residence          0
duration                   0
other_debtors              0
personal                   0
installment_rate           0
present_employment         0
savings                    0
credit_amount              0
purpose                    0
credit_history             0
customer_type              0
dtype: int64

## 查看特征数据类型

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   checking_account_status  1000 non-null   object 
 1   duration                 1000 non-null   int64  
 2   credit_history           1000 non-null   object 
 3   purpose                  1000 non-null   object 
 4   credit_amount            1000 non-null   float64
 5   savings                  1000 non-null   object 
 6   present_employment       1000 non-null   object 
 7   installment_rate         1000 non-null   float64
 8   personal                 1000 non-null   object 
 9   other_debtors            1000 non-null   object 
 10  present_residence        1000 non-null   float64
 11  property                 1000 non-null   object 
 12  age                      1000 non-null   float64
 13  other_installment_plans  1000 non-null   object 
 14  housing                  

## 对分类数据和数值数据进行划分

In [5]:
col_cate = ['checking_account_status','credit_history','purpose','savings','present_employment',
'personal','other_debtors','property','other_installment_plans','housing','job',
'telephone','foreign_worker']  #分类数据
col_num =['duration','credit_amount','installment_rate','present_residence','age',
'existing_credits','dependents','customer_type'] #数值数据

## 对分类数据编码
采用LabelEncoder编码，效果简单直接

In [6]:
for k in col_cate:
    le = LabelEncoder()
    data[k] = le.fit_transform(data[k]) #分类数据编码

## 预览数据和描述性统计

In [7]:
data.head()
data.describe()

,checking_account_status,duration,credit_history,purpose,credit_amount,savings,present_employment,installment_rate,personal,other_debtors,...,property,age,other_installment_plans,housing,existing_credits,job,dependents,telephone,foreign_worker,customer_type
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.577000,20.903000,2.54500,3.277000,3271.258000,1.105000,2.384000,2.973000,1.68200,0.145000,...,1.358000,35.546000,1.675000,0.929000,1.407000,1.904000,1.155000,0.404000,0.037000,1.300000
std,1.257638,12.058814,1.08312,2.739302,2822.736876,1.580023,1.208306,1.118715,0.70808,0.477706,...,1.050209,11.375469,0.705601,0.531264,0.577654,0.653614,0.362086,0.490943,0.188856,0.458487
min,0.000000,4.000000,0.00000,0.000000,250.000000,0.000000,0.000000,1.000000,0.00000,0.000000,...,0.000000,19.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,0.000000,12.000000,2.00000,1.000000,1365.500000,0.000000,2.000000,2.000000,1.00000,0.000000,...,0.000000,27.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,1.000000
50%,1.000000,18.000000,2.00000,3.000000,2319.500000,0.000000,2.000000,3.000000,2.00000,0.000000,...,1.000000,33.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,1.000000
75%,3.000000,24.000000,4.00000,4.000000,3972.250000,2.000000,4.000000,4.000000,2.00000,0.000000,...,2.000000,42.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,0.000000,2.000000
max,3.000000,72.000000,4.00000,9.000000,18424.000000,4.000000,4.000000,4.000000,3.00000,2.000000,...,3.000000,75.000000,2.000000,2.000000,4.000000,3.000000,2.000000,1.000000,1.000000,2.000000


In [8]:
x =data.iloc[:,0:19].values
y=data.iloc[:,20].values #输出


In [29]:
from sklearn.model_selection import GridSearchCV
mlp_clf__tuned_parameters = {"hidden_layer_sizes": [(100,),(50,),(150,),(80,)],
                             "solver": ['adam', 'sgd', 'lbfgs'],
                             "max_iter": [1500],
                             "verbose": [True],
                             "activation":['identity', 'logistic', 'tanh', 'relu'],
                             }
mlp = MLPClassifier()
estimator = GridSearchCV(mlp, mlp_clf__tuned_parameters, n_jobs=6)
estimator.fit(x, y)
print ("best:",estimator.get_params().keys())
print (estimator.best_params_)

In [10]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3) #划分训练集，测试集

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

StandardScaler()

In [12]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [28]:

from sklearn.metrics import classification_report,confusion_matrix

best_score =0;
for i in range(60):
    mlp = MLPClassifier(hidden_layer_sizes=(100,),max_iter=1000,activation='logistic',solver='lbfgs',verbose=True,alpha=0.01)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_test)
    score=mlp.score(X_test, y_test)
    if(score > best_score):
        best_score=score
print(confusion_matrix(y_test,predictions))
print(best_score)

[[169  33]
 [ 55  43]]
0.7366666666666667


In [14]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.73      0.80      0.76       202
           2       0.49      0.40      0.44        98

    accuracy                           0.67       300
   macro avg       0.61      0.60      0.60       300
weighted avg       0.65      0.67      0.66       300

